In [1]:
import pandas as pd
import re

from datetime import datetime

In [2]:
month_dict = {
    'enero': '01', 'febrero': '02', 'marzo': '03', 'abril': '04',
    'mayo': '05', 'junio': '06', 'julio': '07', 'agosto': '08',
    'septiembre': '09', 'octubre': '10', 'noviembre': '11', 'diciembre': '12'
}

def interventions(text):
    pattern = r'([A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]+?:\s*[\s\S]*?)(?=[A-ZÁÉÍÓÚÑ][A-ZÁÉÍÓÚÑ\s]+?:|\Z)'
    interventions = re.findall(pattern, text)
    return interventions


def president_interventio(text):
    match = 'PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR:'
    filtered_list = [elem for elem in text if elem.startswith(match)]
    return filtered_list


def convert_to_iso(date_str):
    parts = date_str.split()
    month = month_dict.get(parts[0].lower(), '01')
    day = parts[1].replace(',', '')
    year = parts[2]
    return f"{year}-{month}-{day.zfill(2)}"



## Discourses

In [3]:
json_file = '../data/raw/discourse.json'

discourses = pd.read_json(json_file)

discourses['timestamp'] = discourses['entry-date'].apply(convert_to_iso)

discourses['interventions'] = discourses['entry-content'].apply(interventions)

discourses['president'] = discourses['interventions'].apply(president_interventio)


## Interventions

In [4]:
interventions = discourses.explode(
    'interventions', ignore_index=True
)
interventions['speaker'] = interventions['interventions'].astype(str).apply(
    lambda x: x.split(':', 1)[0] if len(x.split(':', 1)) > 1 else x)
interventions['speech'] = interventions['interventions'].astype(str).apply(
    lambda x: x.split(':', 1)[1] if len(x.split(':', 1)) > 1 else x
)
interventions.drop(
    columns=['interventions', 'entry-date', 'president', 'entry-title', 'entry-content', 'entry-url'], 
    inplace=True
)

In [5]:
interventions.sample(10)

,timestamp,speaker,speech
48024,2021-08-03,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"Muchas gracias.\nYa se nos pasó, nada más nos..."
105897,2024-03-05,ROSA ICELA RODRÍGUEZ VELÁZQUEZ,"Sí, vuelvo a insistir lo que acabo de informa..."
74873,2022-12-06,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,Sí. Son como testigos de cómo se ha hundido l...
78231,2023-01-25,INTERLOCUTOR,"Muchas gracias, señor presidente, me pongo de..."
76298,2022-12-26,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"Diecisiete años sin condena. ¿Qué, no habla l..."
74995,2022-12-07,INTERVENCIÓN,(inaudible)\n
77171,2023-01-09,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,"Muy bien. Vamos. Pues, los cuatro y los tres,..."
29346,2020-07-29,PRESIDENTE ANDRÉS MANUEL LÓPEZ OBRADOR,Yo nada más en lo general les digo que nunca ...
48617,2021-08-11,INTERLOCUTORA,¿Y esa no sería la misma situación que los pr...
85542,2023-05-12,INTERLOCUTOR,"Mire, usted, que habla de dignidad y creo que..."


In [6]:
interventions.to_json('../data/raw/interventions.json')